In [16]:
#PREPROCESS_make_correlation_ready_profile_V2
#Preperation for the corrleation calculation between metabolites and patientinformation/other omics
#
#[2] make profile of 80 samples

#UPDATE: 23.01.24
#make profile for 40 (acpa-) and 40 (acpa+) samples

import pandas as pd
from scipy import stats
import statistics
import numpy as np

In [22]:
def make_data_dict(data_file):
    
    data_dict = {}
    
    data_df = pd.read_csv(data_file, sep="\t", index_col=0)
    r, c = data_df.shape
    
    feature_list = data_df.index.values
    patientID_list = data_df.columns.values

    for i in range(r):
        feature = feature_list[i]
         
        for j in range(c):
            patientID = patientID_list[j]            
            value = data_df.iloc[i][j]
            data_dict[feature, patientID] = value
    
    return data_dict, feature_list, list(patientID_list)

def main(patient_info_file, data_dict, feature_list, output_file, acpa_status):
    
    correlation_dict = {}

    patient_info_df = pd.read_csv(patient_info_file, sep="\t")
    patient_info_df = patient_info_df[patient_info_df['acpa'] == acpa_status] #considering RA samples only; because healthy samples do not have full information.
    patientID_list = patient_info_df["sample_ID"]

    clinical_covariate_list = ["crp","esr","das28crp", "rf_tit", "acpa_tit"]
    
    for feature in feature_list:
        feature_value_list = [] #this is set1 that will be used for correlation
        for patientID in patientID_list:
            value = data_dict[feature, str(int(patientID))]
            feature_value_list.append(value)
        
        for clinical_covariate in clinical_covariate_list:
            clinical_covariate_value_list = [] #this is set2 that will be used for correlation
            for patientID in patientID_list:

                value = patient_info_df[clinical_covariate].loc[patient_info_df["sample_ID"] == patientID].values[0]
                clinical_covariate_value_list.append(value)

            corr, pval = stats.spearmanr(feature_value_list, clinical_covariate_value_list, nan_policy="omit")
            correlation_dict[feature, clinical_covariate] = [corr, pval]

    #write output
    output_txt = open(output_file,'w')
    for clinical_covariate in clinical_covariate_list:
        output_txt.write('\t%s' % clinical_covariate)
    output_txt.write('\n')
    
    for feature in feature_list:
        output_txt.write(feature)
        for clinical_covariate in clinical_covariate_list:
            corr = correlation_dict[feature, clinical_covariate][0]
            pvalue = correlation_dict[feature, clinical_covariate][1]
            
            if pvalue < 0.01: #Note before I forget: March21.2024: Previous threshold 0.001, ver.dbc was 0.01 
                output_txt.write('\t%s' % corr)

            else:
                output_txt.write('\t0')
        output_txt.write('\n')
    output_txt.close()

    
    

In [23]:
def main_full(patient_info_file, data_dict, feature_list, output_file, acpa_status):
    
    correlation_dict = {}

    patient_info_df = pd.read_csv(patient_info_file, sep="\t")
    patient_info_df = patient_info_df[patient_info_df['acpa'] == acpa_status] #considering RA samples only; because healthy samples do not have full information.
    patientID_list = patient_info_df["sample_ID"]

    clinical_covariate_list = ["crp","esr","das28crp", "rf_tit", "acpa_tit"]
    
    for feature in feature_list:
        feature_value_list = [] #this is set1 that will be used for correlation
        for patientID in patientID_list:
            value = data_dict[feature, str(int(patientID))]
            feature_value_list.append(value)
        
        for clinical_covariate in clinical_covariate_list:
            clinical_covariate_value_list = [] #this is set2 that will be used for correlation
            for patientID in patientID_list:

                value = patient_info_df[clinical_covariate].loc[patient_info_df["sample_ID"] == patientID].values[0]
                clinical_covariate_value_list.append(value)

            corr, pval = stats.spearmanr(feature_value_list, clinical_covariate_value_list, nan_policy="omit")
            correlation_dict[feature, clinical_covariate] = [corr, pval]

    #write output
    output_txt = open(output_file,'w')
    for clinical_covariate in clinical_covariate_list:
        output_txt.write('\t%s' % clinical_covariate)
    output_txt.write('\n')
    
    for feature in feature_list:
        output_txt.write(feature)
        for clinical_covariate in clinical_covariate_list:
            corr = correlation_dict[feature, clinical_covariate][0]
            pvalue = correlation_dict[feature, clinical_covariate][1]
            
            output_txt.write('\t%s' % corr)

        output_txt.write('\n')
    output_txt.close()

In [24]:
#load files
patient_info_file = "../../../preprocessed_data/meta/patient_info_for_statistics.v3.tsv"

aa_file = "../../../preprocessed_data/autoantibody/sengenics_qnorm_data.v2.tsv"
aa_data_dict, aa_feature_list, patientID_list = make_data_dict(aa_file)


In [25]:
#correlation between proteins and clinical information
output_file = "../../../analysis/statistics/autoantibody_correlation_v2/autoantibody_patient_info_correlation.acpa_pos.tsv"
main(patient_info_file, aa_data_dict, aa_feature_list, output_file, 1)

output_file = "../../../analysis/statistics/autoantibody_correlation_v2/autoantibody_patient_info_correlation.acpa_pos.full.tsv"
main_full(patient_info_file, aa_data_dict, aa_feature_list, output_file, 1)


output_file = "../../../analysis/statistics/autoantibody_correlation_v2/autoantibody_patient_info_correlation.acpa_neg.tsv"
main(patient_info_file, aa_data_dict, aa_feature_list, output_file, 2)

output_file = "../../../analysis/statistics/autoantibody_correlation_v2/autoantibody_patient_info_correlation.acpa_neg.full.tsv"
main_full(patient_info_file, aa_data_dict, aa_feature_list, output_file, 2)
